Hierarchical evaluation
========

In [ ]:
import numpy as np
import pandas as pd
import glob
import os.path
import sys

from svecon.HierarchicalGridSearchCV import HierarchicalGridSearchCV
from svecon.EmptyTransformer import EmptyTransformer

from sklearn.cross_validation import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer, StandardScaler, LabelEncoder

from metric_learn import LMNN, NCA, LFDA, Covariance, CMAES
from metric_learn import ITML_Supervised, SDML_Supervised, LSML_Supervised, RCA_Supervised

import plotly
plotly.tools.set_credentials_file(username='sveco', api_key='8701ghzf0i')
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode()

datasetsDirectory = 'datasets'
resultsDirectory = 'datasets-results-all'
dumpsDirectory = 'datasets-results-dumps'

if not os.path.exists(resultsDirectory):
    os.makedirs(resultsDirectory)

if not os.path.exists(dumpsDirectory):
    os.makedirs(dumpsDirectory)

default_n_jobs = 8
default_random_state = 789
default_n_folds = 10
default_shuffle = True

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.handlers = []
fh = logging.FileHandler("{}/error.log".format(resultsDirectory))
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)
fh = logging.StreamHandler(sys.stdout)
fh.setLevel(logging.ERROR)
logger.addHandler(fh)

import pickle
def save_object(obj, filename):
    with open("{}/dump_{}.bin".format(dumpsDirectory,filename), 'wb') as output:
        logger.info("saving file" + filename)
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
cv_per_dataset = {}
def evaluateClassifier(X, y, pipeline, parameters, name=None, datasetName=None):
    
    if datasetName in cv_per_dataset:
        cv = cv_per_dataset[datasetName]
    else:
        cv_per_dataset[datasetName] = cv = \
            StratifiedKFold(y, n_folds=default_n_folds, shuffle=default_shuffle, random_state=default_random_state)
    
    grid_search = HierarchicalGridSearchCV(pipeline, parameters, n_jobs=default_n_jobs, verbose=4, cv=cv)
    grid_search.fit(X, y)
    
    save_object(grid_search, '{}_{}'.format(name,datasetName))
    
    stats = [{
        **x,
        **x['scores'],
        **x['train_scores'],
        **x['params'],
        **x['times'],
     } for x in grid_search.grid_scores_ ]

    for i in stats:
        i.pop('scores')
        i.pop('train_scores')
        i.pop('params')
        i.pop('times')
            
    df = pd.DataFrame(stats)
    df['technique'] = pd.Series([name]*df.shape[0], index=df.index)
    df['dataset'] = pd.Series([datasetName]*df.shape[0], index=df.index)
    
    return df

def evaluatePipeline(X,y,datasetName,pipeline):
    resFilename = '{}/{}_{}_result.csv'.format(resultsDirectory,datasetName,pipeline.__name__[8:])
    
    if os.path.isfile(resFilename):
        logging.info("\t`{}` using `{}` already finished, skipping".format(datasetName,pipeline.__name__[8:]))
        return None
    
    logging.info("\t`{}` using `{}` started".format(datasetName,pipeline.__name__[8:]))
    res = pipeline(X,y,datasetName)
    res.to_csv(resFilename)
    logging.info("\t`{}` using `{}` finished".format(datasetName,pipeline.__name__[8:]))

def noConstraints(Y):
    c = len(set(Y))
    return (10*c*c, 20*c*c, 40*c*c, 80*c*c)

In [ ]:
defaultKnnParams = {
    'knn__n_neighbors': (1, 2, 4, 8, 16, 32, 64, 128),
}
defaultIters = (50, 250, 500, 1000)
defaultImputer = Imputer(missing_values='NaN', strategy='mean', axis=0, verbose=0, copy=False)
defaultStandardizer = StandardScaler(copy=False, with_mean=True, with_std=True)

paramsCmaes = {
    'cmaes__metric': ('diagonal', 'full'),
    'cmaes__n_gen': (100, 250, 1000),
    'cmaes__n_neighbors': (1, 4, 8, 16),
    'cmaes__knn_weights': ('uniform', 'distance'),
#     train_subset_size=1.0,
#     split_size=0.33,
}

paramsLmnn = {
    'lmnn__k': (1, 2, 4, 8, 16, 32),
    'lmnn__regularization': (.1, .5, .9),
    'lmnn__max_iter': defaultIters,
    'lmnn__learn_rate': (1e-7,)#, 1e-8, 1e-9),
}

paramsItml = {
#     'itml__num_constraints': (10, 100, 1000, 10000),
    'itml__gamma': (.01, .1, 1., 10.),
    'itml__max_iters': defaultIters,
}

paramsSdml = {
    'sdml__num_constraints': (10000, 100000),
    'sdml__use_cov': (True, False),
    'sdml__balance_param': (0.1, .25, .5, .75, 1),
    'sdml__sparsity_param': (.01, .05, .1, .25)
}

paramsLsml = {
#     'lsml__num_constraints': (100, 1000, 10000, 100000),
    'lsml__max_iter': defaultIters,
}

paramsNca = {
    'nca__max_iter': defaultIters,
    'nca__learn_rate': (0.1, 0.01),
}

paramsLfda = {
    'lfda__metric': ('weighted', 'orthonormalized'),
}

paramsRca = {
    'rca__num_chunks': (10, 50, 100, 500, 1000),
    'rca__chunk_size': (1, 2, 3, 5, 7, 10, 16, 32),
}

def pipelineKnn(X,y,datasetName):
    pipeline = [
        Pipeline([ ('imputer', defaultImputer), ]),
        Pipeline([ ('empty', EmptyTransformer()), ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, {}, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "kNN", datasetName)

def pipelineCovKnn(X,y,datasetName):
    pipeline = [
        Pipeline([ ('imputer', defaultImputer), ]),
        Pipeline([ ('cov', Covariance()), ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, {}, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "Cov+kNN", datasetName)

def pipelineCmaesKnn(X,y,datasetName):
    pipeline = [
        Pipeline([ ('imputer', defaultImputer), ]),
        Pipeline([ ('cmaes', CMAES(verbose=False)) ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, paramsCmaes, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "CMAES+kNN", datasetName)

def pipelineLmnnKnn(X,y,datasetName):
    pipeline = [
        Pipeline([ ('imputer', defaultImputer), ]),
        Pipeline([ ('lmnn', LMNN(k=3, min_iter=50, max_iter=1000, learn_rate=1e-07, regularization=0.5, convergence_tol=0.001)) ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, paramsLmnn, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "LMNN+kNN", datasetName)


def pipelineItmlKnn(X,y,datasetName):
    pipeline = [
        Pipeline([ ('imputer', defaultImputer), ]),
        Pipeline([ ('itml', ITML_Supervised(gamma=1.,max_iters=1000,convergence_threshold=1e-3,num_constraints=None,bounds=None,A0=None)) ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, {**paramsItml, 'itml__num_constraints':noConstraints(y)}, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "ITML+kNN", datasetName)


def pipelineSdmlKnn(X,y,datasetName):
    pipeline = [
        Pipeline([ ('imputer', defaultImputer), ]),
        Pipeline([ ('sdml', SDML_Supervised(balance_param=0.5, sparsity_param=0.01, use_cov=True, num_constraints=None)), ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, paramsSdml, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "SDML+kNN", datasetName)


def pipelineLsmlKnn(X,y,datasetName):
    pipeline = [
        Pipeline([ ('imputer', defaultImputer), ]),
        Pipeline([ ('lsml', LSML_Supervised(tol=1e-3, max_iter=1000, prior=None, num_constraints=None)), ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, {**paramsLsml, 'lsml__num_constraints':noConstraints(y)}, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "LSML+kNN", datasetName)


def pipelineNcaKnn(X,y,datasetName):
    pipeline = [
        Pipeline([ ('imputer', defaultImputer), ]),
        Pipeline([ ('nca', NCA(max_iter=100, learning_rate=0.01)), ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, paramsNca, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "NCA+kNN", datasetName)


def pipelineLfdaKnn(X,y,datasetName):
    pipeline = [
        Pipeline([ ('imputer', defaultImputer), ]),
        Pipeline([ ('lfda', LFDA(dim=None, k=7, metric='weighted')), ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, {**paramsLfda, 'lfda__k': tuple(range(1, X.shape[1]))}, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "LFDA+kNN", datasetName)


def pipelineRcaKnn(X,y,datasetName):
    pipeline = [
        Pipeline([ ('imputer', defaultImputer), ]),
        Pipeline([ ('rca', RCA_Supervised(dim=None, num_chunks=100, chunk_size=2)), ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, paramsRca, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "RCA+kNN", datasetName)






def pipelineStandKnn(X,y,datasetName):
    pipeline = [
        Pipeline([
            ('imputer', defaultImputer),
            ('standardizer', defaultStandardizer),
        ]),
        Pipeline([ ('empty', EmptyTransformer()), ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, {}, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "stand+kNN", datasetName)

def pipelineStandCovKnn(X,y,datasetName):
    pipeline = [
        Pipeline([
            ('imputer', defaultImputer),
            ('standardizer', defaultStandardizer),
        ]),
        Pipeline([ ('cov', Covariance()), ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, {}, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "stand+Cov+kNN", datasetName)

def pipelineStandCmaesKnn(X,y,datasetName):
    pipeline = [
        Pipeline([
            ('imputer', defaultImputer),
            ('standardizer', defaultStandardizer),
        ]),
        Pipeline([ ('cmaes', CMAES(verbose=False)) ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, paramsCmaes, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "stand+CMAES+kNN", datasetName)

def pipelineStandLmnnKnn(X,y,datasetName):
    pipeline = [
        Pipeline([
            ('imputer', defaultImputer),
            ('standardizer', defaultStandardizer),
        ]),
        Pipeline([ ('lmnn', LMNN(k=3, min_iter=50, max_iter=1000, learn_rate=1e-07, regularization=0.5, convergence_tol=0.001)) ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, paramsLmnn, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "stand+LMNN+kNN", datasetName)


def pipelineStandItmlKnn(X,y,datasetName):
    pipeline = [
        Pipeline([
            ('imputer', defaultImputer),
            ('standardizer', defaultStandardizer),
        ]),
        Pipeline([ ('itml', ITML_Supervised(gamma=1.,max_iters=1000,convergence_threshold=1e-3,num_constraints=None,bounds=None,A0=None)) ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, {**paramsItml, 'itml__num_constraints':noConstraints(y)}, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "stand+ITML+kNN", datasetName)


def pipelineStandSdmlKnn(X,y,datasetName):
    pipeline = [
        Pipeline([
            ('imputer', defaultImputer),
            ('standardizer', defaultStandardizer),
        ]),
        Pipeline([ ('sdml', SDML_Supervised(balance_param=0.5, sparsity_param=0.01, use_cov=True, num_constraints=None)), ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, paramsSdml, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "stand+SDML+kNN", datasetName)


def pipelineStandLsmlKnn(X,y,datasetName):
    pipeline = [
        Pipeline([
            ('imputer', defaultImputer),
            ('standardizer', defaultStandardizer),
        ]),
        Pipeline([ ('lsml', LSML_Supervised(tol=1e-3, max_iter=1000, prior=None, num_constraints=None, verbose=False)), ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, {**paramsLsml, 'lsml__num_constraints':noConstraints(y)}, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "stand+LSML+kNN", datasetName)


def pipelineStandNcaKnn(X,y,datasetName):
    pipeline = [
        Pipeline([
            ('imputer', defaultImputer),
            ('standardizer', defaultStandardizer),
        ]),
        Pipeline([ ('nca', NCA(max_iter=100, learning_rate=0.01)), ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, paramsNca, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "stand+NCA+kNN", datasetName)


def pipelineStandLfdaKnn(X,y,datasetName):
    pipeline = [
        Pipeline([
            ('imputer', defaultImputer),
            ('standardizer', defaultStandardizer),
        ]),
        Pipeline([ ('lfda', LFDA(dim=None, k=7, metric='weighted')), ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, {**paramsLfda, 'lfda__k': tuple(range(1, X.shape[1]))}, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "stand+LFDA+kNN", datasetName)


def pipelineStandRcaKnn(X,y,datasetName):
    pipeline = [
        Pipeline([
            ('imputer', defaultImputer),
            ('standardizer', defaultStandardizer),
        ]),
        Pipeline([ ('rca', RCA_Supervised(dim=None, num_chunks=100, chunk_size=2)), ]),
        Pipeline([ ('knn', KNeighborsClassifier()), ]),
    ]
    params = [ {}, paramsRca, defaultKnnParams, ]
    return evaluateClassifier(X, y, pipeline, params, "stand+RCA+kNN", datasetName)

In [ ]:
import glob, os

datasets = []
for file in glob.glob("{}/*.csv".format(datasetsDirectory)):
    datasets.append(file)
datasets.sort()

datasets.remove('datasets/soybean-large.csv')

# datasets = datasets[4:5]
logging.info("Datasets: " + str(datasets))

for x in datasets:
    print(x, pd.read_csv(x, sep=',', skiprows=1, header=0).shape)

In [ ]:
for filename in datasets:
    results = []
    datasetName = filename[len(datasetsDirectory)+1:-4]
    
    logging.info("Starting `{}` dataset".format(datasetName))

    data = pd.read_csv(filename, sep=',', skiprows=1, header=0)

    y = data['class']
    X = data.drop(['class'], axis=1).values
    
    le = LabelEncoder()
    y = le.fit_transform(y)
    
#     known_label_idx, = np.where(y >= 0)
#     known_labels = y[known_label_idx]
#     uniq, lookup = np.unique(known_labels, return_inverse=True)
#     all_inds = [set(np.where(lookup==c)[0]) for c in range(len(uniq))]
#     print(len(all_inds)-1)
#     print(np.random.randint(0, high=len(all_inds)-1))

#     evaluatePipeline( X,y,datasetName,pipelineKnn )
#     evaluatePipeline( X,y,datasetName,pipelineCovKnn )
    evaluatePipeline( X,y,datasetName,pipelineCmaesKnn )
# #     evaluatePipeline( X,y,datasetName,pipelineLmnnKnn )
#     evaluatePipeline( X,y,datasetName,pipelineItmlKnn )
#     evaluatePipeline( X,y,datasetName,pipelineSdmlKnn )
#     evaluatePipeline( X,y,datasetName,pipelineLsmlKnn )
# #     evaluatePipeline( X,y,datasetName,pipelineNcaKnn )
#     evaluatePipeline( X,y,datasetName,pipelineLfdaKnn )
#     evaluatePipeline( X,y,datasetName,pipelineRcaKnn ) 

#     evaluatePipeline( X,y,datasetName,pipelineStandKnn )
#     evaluatePipeline( X,y,datasetName,pipelineStandCovKnn )
    evaluatePipeline( X,y,datasetName,pipelineStandCmaesKnn )
# #     evaluatePipeline( X,y,datasetName,pipelineStandLmnnKnn )
#     evaluatePipeline( X,y,datasetName,pipelineStandItmlKnn )
#     evaluatePipeline( X,y,datasetName,pipelineStandSdmlKnn )
#     evaluatePipeline( X,y,datasetName,pipelineStandLsmlKnn )
# #     evaluatePipeline( X,y,datasetName,pipelineStandNcaKnn )
#     evaluatePipeline( X,y,datasetName,pipelineStandLfdaKnn )
#     evaluatePipeline( X,y,datasetName,pipelineStandRcaKnn ) 

In [ ]:
# with open('{}/{}'.format(dumpsDirectory, 'dump_Cov+kNN_balance-scale.bin'), 'rb') as input:
#     h = pickle.load(input)

# print(h.steps[-2][-1][0].steps[0][1].M)



Statistics
===========

In [ ]:
resultsMask = "{}/*_result.csv".format(resultsDirectory)
alldatatable = pd.concat([pd.DataFrame.from_csv(x) for x in sorted(glob.glob(resultsMask))])

# BUGFIX - ZERO SCORES
# scores = [ x for x in alldatatable.columns.values if x[:5]=='score']
# alldatatable['mean'] = alldatatable[scores].mean(axis=1)

alldata = alldatatable.set_index(['dataset','technique', alldatatable.index])
alldata

# Best run successrate per dataset

In [ ]:
showBestParams = False
for dataset in alldata.index.levels[0]:
    traces = []
    means = []
    techniques = []
    for technique in alldata.index.levels[1]:
        techniqueRows = alldata.loc[dataset].loc[technique]
        if techniqueRows.shape[0]==0:
            continue
        if technique[:5] == 'stand':
            continue
            
        bestRowNum = techniqueRows['mean'].argmax()
        bestRow = techniqueRows.iloc[bestRowNum,:]
        
        scores = [ x for x in alldata.columns.values if x[:5]=='score']
        train_scores = [ x for x in alldata.columns.values if x[:11]=='train_score']
        if sum(bestRow[scores].values)==0: continue
            
        means.append( bestRow['mean'] ) # if bestRow['mean']>0 else np.median(bestRow[scores].values)
        means.append( bestRow['train_mean'] )
        techniques.append( technique )
        techniques.append( technique+' [train]' )
        traces.append(
            go.Box(
                y=[x for x in bestRow[scores].values if x > 0],
                name=technique,
            )
        )
        traces.append(
            go.Box(
                y=[x for x in bestRow[train_scores].values if x > 0],
                name=technique+' [train]',
            )
        )
        
        if showBestParams:
            print("======",dataset,"======",technique,"======")
            print(bestRow)
            print()
        
    traces.append( go.Scatter( x=techniques, y=means, mode='lines', name='mean', marker=dict(color='black') ))
        
    layout = go.Layout(
        yaxis=dict(title='successrate', zeroline=False ),
        title=dataset,
    )
    fig = go.Figure(data=traces, layout=layout)
    py.iplot(fig)
#     plotly.plotly.image.save_as(fig, filename='{}/{}-plot.png'.format(resultsDirectory, dataset))


# Average success rate across all datasets (from best runs only!)

In [ ]:
valuesPerTechnique = {}
for dataset in alldata.index.levels[0]:
    for technique in alldata.index.levels[1]:
        techniqueRows = alldata.loc[dataset].loc[technique]
        if techniqueRows.shape[0]==0:
            continue
            
        bestRowNum = techniqueRows['mean'].argmax()
        bestRow = techniqueRows.iloc[bestRowNum,:]
        scores = [ x for x in alldata.columns.values if x[:5]=='score']
        
        if technique not in valuesPerTechnique:
            valuesPerTechnique[technique] = {'means':[], 'scores':[]}
        
        # SKIP ALL WHERE THERE IS SUCCESSRATE 0 IN ANY FOLD
        if any(x == 0 for x in bestRow[scores].values):
            continue
        
        valuesPerTechnique[technique]['means'].append(bestRow['mean'])
        valuesPerTechnique[technique]['scores'].extend(bestRow[scores].values)

traces = []
means = []
for technique,value in sorted(valuesPerTechnique.items()):
    traces.append( go.Box(
            y=valuesPerTechnique[technique]['scores'],
            name=technique,
    ))
    means.append((technique, np.mean(valuesPerTechnique[technique]['means'])))
    
traces.append( go.Scatter( 
        x=[x[0] for x in means],
        y=[x[1] for x in means],
        mode='lines', name='mean', marker=dict(color='black')
    ))
    
layout = go.Layout(
    yaxis=dict(title='successrate', zeroline=False ),
    title='Aggregated',
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig)
#     plotly.plotly.image.save_as(fig, filename='{}/{}-plot.png'.format(resultsDirectory, dataset))


# Times for best run per dataset

In [ ]:
for dataset in alldata.index.levels[0]:
    
    traces = []
    for timeCol in [ x for x in alldata.columns.values if x[:4]=='time']:
        times = []
        for technique in alldata.index.levels[1]:
            techniqueRows = alldata.loc[dataset].loc[technique]
            if techniqueRows.shape[0]==0:
                continue
                
            bestRowNum = techniqueRows['mean'].argmax()
            bestRow = techniqueRows.iloc[bestRowNum,:]
            
            times.append( bestRow[timeCol] )

        times = [0 if np.isnan(x) else x for x in times]
            
        traces.append(
            go.Bar(
                y=times,
                x=alldata.index.levels[1],
                name=timeCol
            )
        )
        
    layout = go.Layout(
        barmode='stack',
        yaxis=dict(
            title='seconds',
            zeroline=False
        ),
        title=dataset,
    )
    fig = go.Figure(data=traces, layout=layout)
    py.iplot(fig)
#     plotly.plotly.image.save_as(fig, filename='{}/{}-plot.png'.format(resultsDirectory, dataset))


# Average times per technique per dataset

In [ ]:
for dataset in alldata.index.levels[0]:
    
    traces = []
    for timeCol in [ x for x in alldata.columns.values if x[:4]=='time']:
        times = []
        for technique in alldata.index.levels[1]:
            techniqueRows = alldata.loc[dataset].loc[technique]
            if techniqueRows.shape[0]==0:
                continue
            
            times.append( techniqueRows[timeCol].mean() )

        times = [0 if np.isnan(x) else x for x in times]
            
        traces.append(
            go.Bar(
                y=times,
                x=alldata.index.levels[1],
                name=timeCol
            )
        )
        
    layout = go.Layout(
        barmode='stack',
        yaxis=dict(
            title='seconds',
            zeroline=False
        ),
        title=dataset,
    )
    fig = go.Figure(data=traces, layout=layout)
    py.iplot(fig)
#     plotly.plotly.image.save_as(fig, filename='{}/{}-plot.png'.format(resultsDirectory, dataset))


# Looking at k param (kNN)

In [ ]:
alldataknn = alldatatable.set_index(['dataset','technique', 'knn__n_neighbors', alldatatable.index])
alldataknn

# Success rates for technique per dataset per k param in kNN (a lot of graphs!)

In [ ]:
showBestParams = False
for technique in alldataknn.index.levels[1]: 
    for dataset in alldataknn.index.levels[0]:
        traces = []
        means = []
        techniques = []
        for k in alldataknn.index.levels[2]:
            techniqueRows = alldataknn.loc[dataset].loc[technique].loc[k]            
            
            if techniqueRows.shape[0]==0:
                continue
            bestRowNum = techniqueRows['mean'].argmax()
            bestRow = techniqueRows.loc[bestRowNum,:]

            scores = [ x for x in alldataknn.columns.values if x[:5]=='score']
    #         scores = [ x for x in alldata.columns.values if x[:11]=='train_score']
#             if sum(bestRow[scores].values)==0: continue

            means.append( bestRow['mean'] ) # if bestRow['mean']>0 else np.median(bestRow[scores].values)
    #         means.append( bestRow['train_mean'] )
            techniques.append( "k={}".format(k) )
            traces.append(
                go.Box(
                    y=[x for x in bestRow[scores].values if x > 0],
                    name="k={}".format(k),
                )
            )

            if showBestParams:
                print("======",dataset,"======",technique,"======")
                print(bestRow)
                print()

        traces.append( go.Scatter( x=techniques, y=means, mode='lines', name='mean', marker=dict(color='black') ))

        layout = go.Layout(
            yaxis=dict(title=technique+' successrate', zeroline=False ),
            title=dataset,
        )
        fig = go.Figure(data=traces, layout=layout)
        py.iplot(fig)
    
#     break
# #     plotly.plotly.image.save_as(fig, filename='{}/{}-plot.png'.format(resultsDirectory, dataset))
#     break

# Successrates for each dataset depending on k param (kNN)

In [ ]:
showBestParams = False
for dataset in alldataknn.index.levels[0]:
    traces = []
    for technique in alldataknn.index.levels[1]: 
        if technique[:5]!="stand": continue
        
        means = []
        for k in alldataknn.index.levels[2]:
            techniqueRows = alldataknn.loc[dataset].loc[technique].loc[k]            
            if techniqueRows.shape[0]==0: continue
            bestRowNum = techniqueRows['mean'].argmax()
            bestRow = techniqueRows.loc[bestRowNum,:]
    
            means.append( bestRow['mean'] )

        k_label = ['k='+str(x) for x in alldataknn.index.levels[2]]
        traces.append( go.Scatter( x=k_label, y=means, mode='lines', name=technique ))

    layout = go.Layout(
        yaxis=dict(title='successrate', zeroline=False ),
        title=dataset,
    )
    fig = go.Figure(data=traces, layout=layout)
    py.iplot(fig)
    
# #     plotly.plotly.image.save_as(fig, filename='{}/{}-plot.png'.format(resultsDirectory, dataset))
#     break

# Successrate depending on parameter

In [ ]:
interesting_params = [
    ('lmnn', 'k'),
    ('lmnn', 'max_iter'),
    ('lmnn', 'regularization'),
    ('lmnn', 'max_iter'),
#     ('lmnn', 'learn_rate'),
    ('itml', 'num_constraints'),
    ('itml', 'gamma'),
    ('itml', 'max_iters'),
    ('sdml', 'num_constraints'),
    ('sdml', 'use_cov'),
    ('sdml', 'balance_param'),
    ('sdml', 'sparsity_param'),
    ('lsml', 'num_constraints'),
    ('lsml', 'max_iter'),
    ('nca', 'max_iter'),
    ('nca', 'learn_rate'),
    ('lfda', 'metric'),
#     ('rca', 'num_chunks'),
#     ('rca', 'chunk_size'),
    ('cmaes', 'metric'),
    ('cmaes', 'n_gen'),
    ('cmaes', 'n_neighbors'),
    ('cmaes', 'knn_weights'),
]

for technique,paramName in interesting_params:
    traces = []
    for dataset in alldata.index.levels[0]:
        techniqueFullName = '{}+kNN'.format(technique.upper())

        means = []
        possibleParamValues = alldata['{}__{}'.format(technique, paramName)].dropna().unique()
        for param in possibleParamValues:
            techniqueRows = alldataknn.loc[dataset].loc[techniqueFullName]
            techniqueRows = techniqueRows[techniqueRows['{}__{}'.format(technique, paramName)]==param]
            if techniqueRows.shape[0]==0: continue
            bestRowNum = techniqueRows['mean'].argmax()
            bestRow = techniqueRows.loc[bestRowNum,:]

            means.append( bestRow['mean'] )

        param_labels = ['{}={}'.format(paramName, x) for x in possibleParamValues]
        traces.append( go.Scatter( x=param_labels, y=means, mode='lines', name=dataset ))

    layout = go.Layout(
        yaxis=dict(title='successrate', zeroline=False ),
        title='{} by {}'.format(technique.upper(), paramName),
    )
    fig = go.Figure(data=traces, layout=layout)
    py.iplot(fig)
#     plotly.plotly.image.save_as(fig, filename='{}/{}-plot.png'.format(resultsDirectory, dataset))

# Learning times for all techniques per dataset

In [ ]:
showBestParams = False
traces = []
for technique in alldata.index.levels[1]: 
    times = []
    datasetNames = []
    for dataset in alldata.index.levels[0]:
    
        if technique in ['kNN', 'stand+kNN']: continue
        if dataset in ['mice-protein']: continue
#         if technique[:5]!="stand": continue
        
        techniqueRows = alldataknn.loc[dataset].loc[technique]
        if techniqueRows.shape[0]==0: continue

        bestRowNum = techniqueRows['mean'].argmax()
        bestRow = techniqueRows.loc[bestRowNum,:]

        times.append( bestRow['time2']/3600 )
        datasetNames.append(dataset)
    
    traces.append( go.Scatter( x=datasetNames, y=times, mode='lines', name=technique ))

layout = go.Layout(
    yaxis=dict(title='hours', zeroline=False ),
    title='Learning times',
)
fig = go.Figure(data=traces, layout=layout)
py.iplot(fig)
    
# #     plotly.plotly.image.save_as(fig, filename='{}/{}-plot.png'.format(resultsDirectory, dataset))
#     break

# Training time depending on parameter

In [ ]:
interesting_params = [
    ('lmnn', 'k'),
#     ('lmnn', 'max_iter'),
#     ('lmnn', 'regularization'),
#     ('lmnn', 'max_iter'),
# #     ('lmnn', 'learn_rate'),
#     ('itml', 'num_constraints'),
#     ('itml', 'gamma'),
#     ('itml', 'max_iters'),
#     ('sdml', 'num_constraints'),
#     ('sdml', 'use_cov'),
#     ('sdml', 'balance_param'),
#     ('sdml', 'sparsity_param'),
#     ('lsml', 'num_constraints'),
#     ('lsml', 'max_iter'),
# #     ('nca', 'max_iter'),
# #     ('nca', 'learn_rate'),
#     ('lfda', 'metric'),
# #     ('rca', 'num_chunks'),
# #     ('rca', 'chunk_size'),
#     ('cmaes', 'metric'),
#     ('cmaes', 'n_gen'),
#     ('cmaes', 'n_neighbors'),
#     ('cmaes', 'knn_weights'),
]

for technique,paramName in interesting_params:
    traces = []
    for dataset in alldata.index.levels[0]:
        techniqueFullName = '{}+kNN'.format(technique.upper())

        means = []
        possibleParamValues = alldata['{}__{}'.format(technique, paramName)].dropna().unique()
        for param in possibleParamValues:
            techniqueRows = alldataknn.loc[dataset].loc[techniqueFullName]
            techniqueRows = techniqueRows[techniqueRows['{}__{}'.format(technique, paramName)]==param]
            if techniqueRows.shape[0]==0: continue

            means.append( techniqueRows['time2'].mean() /3600 )

        param_labels = ['{}={}'.format(paramName, x) for x in possibleParamValues]
        traces.append( go.Scatter( x=param_labels, y=means, mode='lines', name=dataset ))

    layout = go.Layout(
        yaxis=dict(title='hours', zeroline=False ),
        title='{} by {}'.format(technique.upper(), paramName),
    )
    fig = go.Figure(data=traces, layout=layout)
    py.iplot(fig)
#     plotly.plotly.image.save_as(fig, filename='{}/{}-plot.png'.format(resultsDirectory, dataset))